In [1]:
import pygame
import random

# --- 1. ARCHITECTURE SETUP ---
WIDTH, HEIGHT = 800, 600
FPS = 60

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Project 3: AI Swarm Simulation")
clock = pygame.time.Clock()

# --- 2. THE AGENT BLUEPRINT ---
class Agent:
    def __init__(self):
        # Start at a random position
        self.x = random.randint(0, WIDTH)
        self.y = random.randint(0, HEIGHT)
        self.color = (0, 255, 150) # Neon Green
        self.size = 5

    def move(self):
        # Basic movement: Just random jittering for Phase 1
        self.x += random.randint(-3, 3)
        self.y += random.randint(-3, 3)
        
        # Keep them inside the screen bounds
        self.x = max(0, min(WIDTH, self.x))
        self.y = max(0, min(HEIGHT, self.y))

    def draw(self):
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.size)

# --- 3. INITIALIZE THE SWARM ---
# Create a list containing 50 Agent objects
my_swarm = [Agent() for _ in range(50)]

# --- 4. THE MAIN SIMULATION LOOP ---
print("Starting Simulation... Close the window to stop.")
running = True

while running:
    # A. Wipe the screen clean every frame (Dark Gray)
    screen.fill((30, 30, 30))

    # B. Listen for the user closing the window
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # C. Update and draw every agent in our swarm
    for agent in my_swarm:
        agent.move()
        agent.draw()

    # D. Refresh the screen
    pygame.display.flip()
    clock.tick(FPS) # Lock to 60 frames per second

pygame.quit()

C:\Users\G15\anaconda3\envs\DL\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.11.14)
Hello from the pygame community. https://www.pygame.org/contribute.html
Starting Simulation... Close the window to stop.


In [2]:
import pygame
import random
import math

# --- 1. ARCHITECTURE SETUP ---
WIDTH, HEIGHT = 800, 600
FPS = 60

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Project 3: Swarm vs Predator")
clock = pygame.time.Clock()

# --- 2. THE UPGRADED AGENT ---
class Agent:
    def __init__(self):
        # Using Vectors for smooth physics (position and velocity)
        self.pos = pygame.math.Vector2(random.randint(0, WIDTH), random.randint(0, HEIGHT))
        self.vel = pygame.math.Vector2(random.uniform(-2, 2), random.uniform(-2, 2))
        
        self.color = (0, 255, 150)
        self.size = 5
        self.max_speed = 4

    def update(self, predator_pos):
        # RULE 1: AVOID THE PREDATOR
        dist_to_predator = self.pos.distance_to(predator_pos)
        
        # If the predator is within 100 pixels, trigger the escape!
        if 0 < dist_to_predator < 100:
            # Calculate the exact opposite direction to flee
            flee_vector = self.pos - predator_pos
            flee_vector.scale_to_length(self.max_speed + 2) # Sprint away!
            self.vel = self.vel.lerp(flee_vector, 0.1) # Smoothly turn away
        else:
            # RULE 2: WANDER NORMALLY
            wiggle = pygame.math.Vector2(random.uniform(-0.5, 0.5), random.uniform(-0.5, 0.5))
            self.vel += wiggle

        # Enforce the speed limit
        if self.vel.length() > self.max_speed:
            self.vel.scale_to_length(self.max_speed)
            
        # Move the agent
        self.pos += self.vel

        # Wrap around the screen edges (Pac-Man style)
        if self.pos.x > WIDTH: self.pos.x = 0
        elif self.pos.x < 0: self.pos.x = WIDTH
        if self.pos.y > HEIGHT: self.pos.y = 0
        elif self.pos.y < 0: self.pos.y = HEIGHT

    def draw(self):
        pygame.draw.circle(screen, self.color, (int(self.pos.x), int(self.pos.y)), self.size)

# --- 3. INITIALIZE 100 AGENTS ---
my_swarm = [Agent() for _ in range(100)]

# --- 4. MAIN LOOP ---
running = True
while running:
    screen.fill((30, 30, 30))
    
    # Get the mouse position to act as the Predator
    mouse_pos = pygame.math.Vector2(pygame.mouse.get_pos())

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Draw the Predator (Red Circle)
    pygame.draw.circle(screen, (255, 50, 50), (int(mouse_pos.x), int(mouse_pos.y)), 15)

    # Update the swarm based on the Predator's location
    for agent in my_swarm:
        agent.update(mouse_pos)
        agent.draw()

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()